In [1]:
# this notebook follows on from more_cleaning_normalisation

In [14]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 500)

In [15]:
import pickle
filename = '../Data/big_cats/processed_data/scaled_data_dtypes_15_09_2021.pkl'
with open(filename, 'rb') as handle:
    data_scaled_dtypes = pickle.load(handle)

In [16]:
filename = '../Data/big_cats/processed_data/scaled_data_15_09_2021.csv'
data_scaled = pd.read_csv(filename, dtype = data_scaled_dtypes)

# Make data trajectories of visits
For each column calculate a linear fit and use the gradient as the feature

In [5]:
data_scaled.sort_values(['ARKS Number', 'Date']).head()

,ARKS Number,Acute kidney failure,Age at Death,CKD - cause of mortality,CKD - co-morbidity,Date of Birth,Infection,Neoplasia,Neurological,Old Age debility,Respiratory/Cardiovascular,Trauma,Date,haem: RBC (x 10^12/L),haem: haemoglobin (g/L),haem: HCT (%),haem: Total Protein (refractometer),haem: leukocyte count (x10^9/L),haem: neutrophils (%),haem: neutrophils (band),haem: neutrophils(seg) (x10^9/L),haem: lymphocytes (x10^9/L),haem: lymphocytes (%),haem: monocytes (x10^9/L),haem: monocytes (%),haem: eosinophils (x10^9/L),haem: eosinophils (%),haem: estimated platelets x 10^9/L (Hi),haem: MCV (fL),haem: MCHC (g/L),haem: MCH (pg),biochem: Low USG?,biochem: BUN (mmol/L),biochem: Hypercreatininaemia?,biochem: Creatinine (umol/L),biochem: Total Plasma Protein (g/L),biochem: Serum Albumin (g/L),biochem: Globulins (g/L),biochem: Anaemic? <25,biochem: Haematocrit (%),biochem: Potassium (mmol/L),biochem: Calcium (mmol/L),biochem: Hyperphosphataemia? >2_9 to 3,biochem: Phosphate (mmol/L),biochem: Sodium (mmol/L),biochem: Chloride (mmol/L),biochem: Bicarbonate,biochem: Glucose,biochem: CK (U/L),biochem: ALP (U/L),biochem: ALT (U/L),biochem: AST,biochem: Amylase,biochem: Cholesterol (mmol/L),biochem: Bilirubin,urine: Urine Specific Gravity,urine: Low USG?,urine: Azotaemia?,urine: Urine protein (g/L),urine: urine creatinine (umol/l),urine: Protein:creatinine ratio,urine: Proteinuric UPC greater 0:4,urine: glucose (Multistix),urine: bilirubin (Multistix),urine: ketones (Multistix),urine: blood (Hb -Multistix),urine: pH,urine: protein (Multistix),urine: Positive urine culture,urine: leukocytes (/cumm)x,urine: blood (RBC/ ul) (haematuriax,urine: Lipid,urine: Casts,urine: Epith cells,urine: Crystals,urine: Organisms,urine: Contaminants?,usg_maxed_out,urine: blood haemolysed,Date dt,Date of Birth dt,age at visit,num visits,num key features missing,age at visit days,Age at Death days,Sex_female,Species (common name)_African Lion,Species (common name)_Asiatic Lion,Species (common name)_Bengal Tiger,Species (common name)_Bengal Tiger (white),Species (common name)_Bobcat,Species (common name)_Caracal,Species (common name)_Cheetah,Species (common name)_Clouded Leopard,Species (common name)_Cougar,Species (common name)_Fishing Cat,Species (common name)_Jaguar,Species (common name)_Jaguarundi,Species (common name)_Leopard,Species (common name)_Leopard Cat,Species (common name)_Lion,Species (common name)_Northern (Canadian) Lynx,Species (common name)_Ocelot,Species (common name)_Persian Leopard,Species (common name)_Serval,Species (common name)_Siberian Tiger,Species (common name)_Snow Leopard,Species (common name)_Sumatran Tiger,Species (common name)_Temminck's Golden Cat,Species (common name)_Tiger,Species (common name)_Tiger (White)
91,200003,False,22y0m19d,False,True,1993-01-01,False,False,False,True,False,False,2010-06-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.08843,0.0,0.091514,0.365854,0.426667,0.4,0.0,0.509091,0.059524,0.496078,0.0,0.08913,0.451613,0.5,0.377049,NaN,0.001453,0.028292,0.060868,0.033739,NaN,0.077632,0.011834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2010-06-25,1993-01-01,6384 days 00:00:00.000000000,5,1,0.677673,0.875342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
92,200003,False,22y0m19d,False,True,1993-01-01,False,False,False,True,False,False,2012-07-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065543,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2012-07-13,1993-01-01,7133 days 00:00:00.000000000,5,2,0.757193,0.875342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
93,200003,False,22y0m19d,False,True,1993-01-01,False,False,False,True,False,False,2013-01-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
cols_we_want = list(data_scaled.select_dtypes(include=['float', 'int']).columns)
cols_we_want.remove('num visits')
cols_we_want.remove('num key features missing')
cols_we_want.remove('Age at Death days')
cols_we_want

['haem: RBC (x 10^12/L)',
 'haem: haemoglobin (g/L)',
 'haem: HCT (%)',
 'haem: Total Protein (refractometer)',
 'haem: leukocyte count (x10^9/L)',
 'haem: neutrophils (%)',
 'haem: neutrophils (band)',
 'haem: neutrophils(seg) (x10^9/L)',
 'haem: lymphocytes (x10^9/L)',
 'haem: lymphocytes  (%)',
 'haem: monocytes (x10^9/L)',
 'haem: monocytes (%)',
 'haem: eosinophils  (x10^9/L)',
 'haem: eosinophils (%)',
 'haem: estimated platelets x 10^9/L (Hi)',
 'haem: MCV (fL)',
 'haem: MCHC (g/L)',
 'haem: MCH (pg)',
 'biochem: Low USG?',
 'biochem: BUN (mmol/L)',
 'biochem: Hypercreatininaemia?',
 'biochem: Creatinine (umol/L)',
 'biochem: Total Plasma Protein (g/L)',
 'biochem: Serum Albumin (g/L)',
 'biochem: Globulins (g/L)',
 'biochem: Anaemic? <25',
 'biochem: Haematocrit (%)',
 'biochem: Potassium (mmol/L)',
 'biochem: Calcium (mmol/L)',
 'biochem: Hyperphosphataemia? >2_9 to 3',
 'biochem: Phosphate (mmol/L)',
 'biochem: Sodium (mmol/L)',
 'biochem: Chloride (mmol/L)',
 'biochem: Bicar

In [7]:
from sklearn.linear_model import LinearRegression

feature_df = pd.DataFrame()
cats = data_scaled['ARKS Number'].unique()


for cat in cats:
    cat_data = data_scaled[data_scaled['ARKS Number'] == cat]
    gradient_dict = {}
    gradient_dict['ARKS Number'] = cat
   
    for col in cols_we_want:
        # drop nans
        cat_data_subset = cat_data.dropna(subset = [col])
        # check for 2 values
        if len(cat_data_subset) >= 2:
            # do the linear fit
            X = cat_data_subset['age at visit days'].values.reshape(-1, 1)
            y = cat_data_subset[col].values.reshape(-1, 1)
            reg = LinearRegression().fit(X, y)
            grad = reg.coef_
            intercept = reg.intercept_
            gradient_dict[col + '_grad'] = grad[0][0]
            gradient_dict[col + '_intercept'] = intercept[0]
            
        else:
            # set gradient to nan
            gradient_dict[col + '_grad'] = np.nan
            gradient_dict[col + '_intercept'] = np.nan
    
    # add this cat to the df
    feature_df = feature_df.append(gradient_dict, ignore_index = True)

In [8]:
feature_df

,ARKS Number,age at visit days_grad,age at visit days_intercept,biochem: ALP (U/L)_grad,biochem: ALP (U/L)_intercept,biochem: ALT (U/L)_grad,biochem: ALT (U/L)_intercept,biochem: AST_grad,biochem: AST_intercept,biochem: Amylase_grad,biochem: Amylase_intercept,biochem: Anaemic? <25_grad,biochem: Anaemic? <25_intercept,biochem: BUN (mmol/L)_grad,biochem: BUN (mmol/L)_intercept,biochem: Bicarbonate_grad,biochem: Bicarbonate_intercept,biochem: Bilirubin_grad,biochem: Bilirubin_intercept,biochem: CK (U/L)_grad,biochem: CK (U/L)_intercept,biochem: Calcium (mmol/L)_grad,biochem: Calcium (mmol/L)_intercept,biochem: Chloride (mmol/L)_grad,biochem: Chloride (mmol/L)_intercept,biochem: Cholesterol (mmol/L)_grad,biochem: Cholesterol (mmol/L)_intercept,biochem: Creatinine (umol/L)_grad,biochem: Creatinine (umol/L)_intercept,biochem: Globulins (g/L)_grad,biochem: Globulins (g/L)_intercept,biochem: Glucose_grad,biochem: Glucose_intercept,biochem: Haematocrit (%)_grad,biochem: Haematocrit (%)_intercept,biochem: Hypercreatininaemia?_grad,biochem: Hypercreatininaemia?_intercept,biochem: Hyperphosphataemia? >2_9 to 3_grad,biochem: Hyperphosphataemia? >2_9 to 3_intercept,biochem: Low USG?_grad,biochem: Low USG?_intercept,biochem: Phosphate (mmol/L)_grad,biochem: Phosphate (mmol/L)_intercept,biochem: Potassium (mmol/L)_grad,biochem: Potassium (mmol/L)_intercept,biochem: Serum Albumin (g/L)_grad,biochem: Serum Albumin (g/L)_intercept,biochem: Sodium (mmol/L)_grad,biochem: Sodium (mmol/L)_intercept,biochem: Total Plasma Protein (g/L)_grad,biochem: Total Plasma Protein (g/L)_intercept,haem: HCT (%)_grad,haem: HCT (%)_intercept,haem: MCH (pg)_grad,haem: MCH (pg)_intercept,haem: MCHC (g/L)_grad,haem: MCHC (g/L)_intercept,haem: MCV (fL)_grad,haem: MCV (fL)_intercept,haem: RBC (x 10^12/L)_grad,haem: RBC (x 10^12/L)_intercept,haem: Total Protein (refractometer)_grad,haem: Total Protein (refractometer)_intercept,haem: eosinophils (x10^9/L)_grad,haem: eosinophils (x10^9/L)_intercept,haem: eosinophils (%)_grad,haem: eosinophils (%)_intercept,haem: estimated platelets x 10^9/L (Hi)_grad,haem: estimated platelets x 10^9/L (Hi)_intercept,haem: haemoglobin (g/L)_grad,haem: haemoglobin (g/L)_intercept,haem: leukocyte count (x10^9/L)_grad,haem: leukocyte count (x10^9/L)_intercept,haem: lymphocytes (%)_grad,haem: lymphocytes (%)_intercept,haem: lymphocytes (x10^9/L)_grad,haem: lymphocytes (x10^9/L)_intercept,haem: monocytes (%)_grad,haem: monocytes (%)_intercept,haem: monocytes (x10^9/L)_grad,haem: monocytes (x10^9/L)_intercept,haem: neutrophils (%)_grad,haem: neutrophils (%)_intercept,haem: neutrophils (band)_grad,haem: neutrophils (band)_intercept,haem: neutrophils(seg) (x10^9/L)_grad,haem: neutrophils(seg) (x10^9/L)_intercept,urine: Azotaemia?_grad,urine: Azotaemia?_intercept,urine: Casts_grad,urine: Casts_intercept,urine: Contaminants?_grad,urine: Contaminants?_intercept,urine: Crystals_grad,urine: Crystals_intercept,urine: Epith cells_grad,urine: Epith cells_intercept,urine: Lipid_grad,urine: Lipid_intercept,urine: Low USG?_grad,urine: Low USG?_intercept,urine: Organisms_grad,urine: Organisms_intercept,urine: Positive urine culture_grad,urine: Positive urine culture_intercept,urine: Protein:creatinine ratio_grad,urine: Protein:creatinine ratio_intercept,urine: Proteinuric UPC greater 0:4_grad,urine: Proteinuric UPC greater 0:4_intercept,urine: Urine Specific Gravity_grad,urine: Urine Specific Gravity_intercept,urine: Urine protein (g/L)_grad,urine: Urine protein (g/L)_intercept,urine: bilirubin (Multistix)_grad,urine: bilirubin (Multistix)_intercept,urine: blood haemolysed_grad,urine: blood haemolysed_intercept,urine: glucose (Multistix)_grad,urine: glucose (Multistix)_intercept,urine: ketones (Multistix)_grad,urine: ketones (Multistix)_intercept,urine: pH_grad,urine: pH_intercept,urine: protein (Multistix)_grad,urine: protein (Multistix)_intercept,urine: urine creatinine (umol/l)_grad,urine: urine creatinine (umol/l)_intercept,usg_maxed_out_grad,usg_maxed_out

### Make demographics df

In [9]:
dem_list = ['ARKS Number',
 'CKD - cause of mortality',
 'Species (common name)_African Lion',
 'Species (common name)_Asiatic Lion',
 'Species (common name)_Bengal Tiger',
 'Species (common name)_Bengal Tiger (white)',
 'Species (common name)_Bobcat',
 'Species (common name)_Caracal',
 'Species (common name)_Cheetah',
 'Species (common name)_Clouded Leopard',
 'Species (common name)_Cougar',
 'Species (common name)_Fishing Cat',
 'Species (common name)_Jaguar',
 'Species (common name)_Jaguarundi',
 'Species (common name)_Leopard',
 'Species (common name)_Leopard Cat',
 'Species (common name)_Lion',
 'Species (common name)_Northern (Canadian) Lynx',
 'Species (common name)_Ocelot',
 'Species (common name)_Persian Leopard',
 'Species (common name)_Serval',
 'Species (common name)_Siberian Tiger',
 'Species (common name)_Snow Leopard',
 'Species (common name)_Sumatran Tiger',
 "Species (common name)_Temminck's Golden Cat",
 'Species (common name)_Tiger',
 'Species (common name)_Tiger (White)']

In [10]:
demographics = data_scaled[dem_list].drop_duplicates()
demographics

,ARKS Number,CKD - cause of mortality,Species (common name)_African Lion,Species (common name)_Asiatic Lion,Species (common name)_Bengal Tiger,Species (common name)_Bengal Tiger (white),Species (common name)_Bobcat,Species (common name)_Caracal,Species (common name)_Cheetah,Species (common name)_Clouded Leopard,Species (common name)_Cougar,Species (common name)_Fishing Cat,Species (common name)_Jaguar,Species (common name)_Jaguarundi,Species (common name)_Leopard,Species (common name)_Leopard Cat,Species (common name)_Lion,Species (common name)_Northern (Canadian) Lynx,Species (common name)_Ocelot,Species (common name)_Persian Leopard,Species (common name)_Serval,Species (common name)_Siberian Tiger,Species (common name)_Snow Leopard,Species (common name)_Sumatran Tiger,Species (common name)_Temminck's Golden Cat,Species (common name)_Tiger,Species (common name)_Tiger (White)
0,Koyukon,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,780127,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,800014MZ,False,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
14,920519,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
20,980010,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
22,980014,False,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,999028,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30,A39042,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32,A50003,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,A79192,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### Join demographcs to linear features

In [11]:
total_df = feature_df.merge(demographics, on='ARKS Number')
total_df

,ARKS Number,age at visit days_grad,age at visit days_intercept,biochem: ALP (U/L)_grad,biochem: ALP (U/L)_intercept,biochem: ALT (U/L)_grad,biochem: ALT (U/L)_intercept,biochem: AST_grad,biochem: AST_intercept,biochem: Amylase_grad,biochem: Amylase_intercept,biochem: Anaemic? <25_grad,biochem: Anaemic? <25_intercept,biochem: BUN (mmol/L)_grad,biochem: BUN (mmol/L)_intercept,biochem: Bicarbonate_grad,biochem: Bicarbonate_intercept,biochem: Bilirubin_grad,biochem: Bilirubin_intercept,biochem: CK (U/L)_grad,biochem: CK (U/L)_intercept,biochem: Calcium (mmol/L)_grad,biochem: Calcium (mmol/L)_intercept,biochem: Chloride (mmol/L)_grad,biochem: Chloride (mmol/L)_intercept,biochem: Cholesterol (mmol/L)_grad,biochem: Cholesterol (mmol/L)_intercept,biochem: Creatinine (umol/L)_grad,biochem: Creatinine (umol/L)_intercept,biochem: Globulins (g/L)_grad,biochem: Globulins (g/L)_intercept,biochem: Glucose_grad,biochem: Glucose_intercept,biochem: Haematocrit (%)_grad,biochem: Haematocrit (%)_intercept,biochem: Hypercreatininaemia?_grad,biochem: Hypercreatininaemia?_intercept,biochem: Hyperphosphataemia? >2_9 to 3_grad,biochem: Hyperphosphataemia? >2_9 to 3_intercept,biochem: Low USG?_grad,biochem: Low USG?_intercept,biochem: Phosphate (mmol/L)_grad,biochem: Phosphate (mmol/L)_intercept,biochem: Potassium (mmol/L)_grad,biochem: Potassium (mmol/L)_intercept,biochem: Serum Albumin (g/L)_grad,biochem: Serum Albumin (g/L)_intercept,biochem: Sodium (mmol/L)_grad,biochem: Sodium (mmol/L)_intercept,biochem: Total Plasma Protein (g/L)_grad,biochem: Total Plasma Protein (g/L)_intercept,haem: HCT (%)_grad,haem: HCT (%)_intercept,haem: MCH (pg)_grad,haem: MCH (pg)_intercept,haem: MCHC (g/L)_grad,haem: MCHC (g/L)_intercept,haem: MCV (fL)_grad,haem: MCV (fL)_intercept,haem: RBC (x 10^12/L)_grad,haem: RBC (x 10^12/L)_intercept,haem: Total Protein (refractometer)_grad,haem: Total Protein (refractometer)_intercept,haem: eosinophils (x10^9/L)_grad,haem: eosinophils (x10^9/L)_intercept,haem: eosinophils (%)_grad,haem: eosinophils (%)_intercept,haem: estimated platelets x 10^9/L (Hi)_grad,haem: estimated platelets x 10^9/L (Hi)_intercept,haem: haemoglobin (g/L)_grad,haem: haemoglobin (g/L)_intercept,haem: leukocyte count (x10^9/L)_grad,haem: leukocyte count (x10^9/L)_intercept,haem: lymphocytes (%)_grad,haem: lymphocytes (%)_intercept,haem: lymphocytes (x10^9/L)_grad,haem: lymphocytes (x10^9/L)_intercept,haem: monocytes (%)_grad,haem: monocytes (%)_intercept,haem: monocytes (x10^9/L)_grad,haem: monocytes (x10^9/L)_intercept,haem: neutrophils (%)_grad,haem: neutrophils (%)_intercept,haem: neutrophils (band)_grad,haem: neutrophils (band)_intercept,haem: neutrophils(seg) (x10^9/L)_grad,haem: neutrophils(seg) (x10^9/L)_intercept,urine: Azotaemia?_grad,urine: Azotaemia?_intercept,urine: Casts_grad,urine: Casts_intercept,urine: Contaminants?_grad,urine: Contaminants?_intercept,urine: Crystals_grad,urine: Crystals_intercept,urine: Epith cells_grad,urine: Epith cells_intercept,urine: Lipid_grad,urine: Lipid_intercept,urine: Low USG?_grad,urine: Low USG?_intercept,urine: Organisms_grad,urine: Organisms_intercept,urine: Positive urine culture_grad,urine: Positive urine culture_intercept,urine: Protein:creatinine ratio_grad,urine: Protein:creatinine ratio_intercept,urine: Proteinuric UPC greater 0:4_grad,urine: Proteinuric UPC greater 0:4_intercept,urine: Urine Specific Gravity_grad,urine: Urine Specific Gravity_intercept,urine: Urine protein (g/L)_grad,urine: Urine protein (g/L)_intercept,urine: bilirubin (Multistix)_grad,urine: bilirubin (Multistix)_intercept,urine: blood haemolysed_grad,urine: blood haemolysed_intercept,urine: glucose (Multistix)_grad,urine: glucose (Multistix)_intercept,urine: ketones (Multistix)_grad,urine: ketones (Multistix)_intercept,urine: pH_grad,urine: pH_intercept,urine: protein (Multistix)_grad,urine: protein (Multistix)_intercept,urine: urine creatinine (umol/l)_grad,urine: urine creatinine (umol/l)_intercept,usg_maxed_out_grad,usg_maxed_out

In [12]:
total_df.describe()

,age at visit days_grad,age at visit days_intercept,biochem: ALP (U/L)_grad,biochem: ALP (U/L)_intercept,biochem: ALT (U/L)_grad,biochem: ALT (U/L)_intercept,biochem: AST_grad,biochem: AST_intercept,biochem: Amylase_grad,biochem: Amylase_intercept,biochem: Anaemic? <25_grad,biochem: Anaemic? <25_intercept,biochem: BUN (mmol/L)_grad,biochem: BUN (mmol/L)_intercept,biochem: Bicarbonate_grad,biochem: Bicarbonate_intercept,biochem: Bilirubin_grad,biochem: Bilirubin_intercept,biochem: CK (U/L)_grad,biochem: CK (U/L)_intercept,biochem: Calcium (mmol/L)_grad,biochem: Calcium (mmol/L)_intercept,biochem: Chloride (mmol/L)_grad,biochem: Chloride (mmol/L)_intercept,biochem: Cholesterol (mmol/L)_grad,biochem: Cholesterol (mmol/L)_intercept,biochem: Creatinine (umol/L)_grad,biochem: Creatinine (umol/L)_intercept,biochem: Globulins (g/L)_grad,biochem: Globulins (g/L)_intercept,biochem: Glucose_grad,biochem: Glucose_intercept,biochem: Haematocrit (%)_grad,biochem: Haematocrit (%)_intercept,biochem: Hypercreatininaemia?_grad,biochem: Hypercreatininaemia?_intercept,biochem: Hyperphosphataemia? >2_9 to 3_grad,biochem: Hyperphosphataemia? >2_9 to 3_intercept,biochem: Low USG?_grad,biochem: Low USG?_intercept,biochem: Phosphate (mmol/L)_grad,biochem: Phosphate (mmol/L)_intercept,biochem: Potassium (mmol/L)_grad,biochem: Potassium (mmol/L)_intercept,biochem: Serum Albumin (g/L)_grad,biochem: Serum Albumin (g/L)_intercept,biochem: Sodium (mmol/L)_grad,biochem: Sodium (mmol/L)_intercept,biochem: Total Plasma Protein (g/L)_grad,biochem: Total Plasma Protein (g/L)_intercept,haem: HCT (%)_grad,haem: HCT (%)_intercept,haem: MCH (pg)_grad,haem: MCH (pg)_intercept,haem: MCHC (g/L)_grad,haem: MCHC (g/L)_intercept,haem: MCV (fL)_grad,haem: MCV (fL)_intercept,haem: RBC (x 10^12/L)_grad,haem: RBC (x 10^12/L)_intercept,haem: Total Protein (refractometer)_grad,haem: Total Protein (refractometer)_intercept,haem: eosinophils (x10^9/L)_grad,haem: eosinophils (x10^9/L)_intercept,haem: eosinophils (%)_grad,haem: eosinophils (%)_intercept,haem: estimated platelets x 10^9/L (Hi)_grad,haem: estimated platelets x 10^9/L (Hi)_intercept,haem: haemoglobin (g/L)_grad,haem: haemoglobin (g/L)_intercept,haem: leukocyte count (x10^9/L)_grad,haem: leukocyte count (x10^9/L)_intercept,haem: lymphocytes (%)_grad,haem: lymphocytes (%)_intercept,haem: lymphocytes (x10^9/L)_grad,haem: lymphocytes (x10^9/L)_intercept,haem: monocytes (%)_grad,haem: monocytes (%)_intercept,haem: monocytes (x10^9/L)_grad,haem: monocytes (x10^9/L)_intercept,haem: neutrophils (%)_grad,haem: neutrophils (%)_intercept,haem: neutrophils (band)_grad,haem: neutrophils (band)_intercept,haem: neutrophils(seg) (x10^9/L)_grad,haem: neutrophils(seg) (x10^9/L)_intercept,urine: Azotaemia?_grad,urine: Azotaemia?_intercept,urine: Casts_grad,urine: Casts_intercept,urine: Contaminants?_grad,urine: Contaminants?_intercept,urine: Crystals_grad,urine: Crystals_intercept,urine: Epith cells_grad,urine: Epith cells_intercept,urine: Lipid_grad,urine: Lipid_intercept,urine: Low USG?_grad,urine: Low USG?_intercept,urine: Organisms_grad,urine: Organisms_intercept,urine: Positive urine culture_grad,urine: Positive urine culture_intercept,urine: Protein:creatinine ratio_grad,urine: Protein:creatinine ratio_intercept,urine: Proteinuric UPC greater 0:4_grad,urine: Proteinuric UPC greater 0:4_intercept,urine: Urine Specific Gravity_grad,urine: Urine Specific Gravity_intercept,urine: Urine protein (g/L)_grad,urine: Urine protein (g/L)_intercept,urine: bilirubin (Multistix)_grad,urine: bilirubin (Multistix)_intercept,urine: blood haemolysed_grad,urine: blood haemolysed_intercept,urine: glucose (Multistix)_grad,urine: glucose (Multistix)_intercept,urine: ketones (Multistix)_grad,urine: ketones (Multistix)_intercept,urine: pH_grad,urine: pH_intercept,urine: protein (Multistix)_grad,urine: protein (Multistix)_intercept,urine: urine creatinine (umol/l)_grad,urine: urine creatinine (umol/l)_intercept,usg_maxed_out_grad,usg_maxed_out_intercept,S

## Write total_df to file

In [13]:
filename = '../Data/big_cats/processed_data/grad_intercept_data_15_09_2021.csv'
total_df.to_csv(filename, index=False)